## Initialization

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow==2.4.0
!pip install keras==2.4.0
!pip install kymatio
!pip install tqdm
!pip install iterative-stratification
!pip install scikit-multilearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement tensorflow==2.4.0 (from versions: 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.5, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.7.2, 2.7.3, 2.7.4, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0)
ERROR: No matching distribution found for tensorflow==2.4.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.2/170.2 KB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 68.0 MB/s eta 0:00:

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 KB 6.0 MB/s eta 0:00:00


## Definitions

In [ ]:
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import train_test_split, KFold
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
import numpy as np
import os
import pickle
import sys
sys.path.append("drive/MyDrive/Scattering_Novo/src")
sys.path.append("drive/MyDrive/DeSpaWN-main")
# Import Data
import scipy.io as sio
from DataHandler import DataHandler
from ModelHandler import ModelHandler
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from skmultilearn.model_selection import iterative_train_test_split
from sklearn.model_selection import KFold
from sklearn.utils.class_weight import compute_class_weight, compute_sample_weight
 
configs = {
    "N_GRIDS": 5, 
    "SIGNAL_BASE_LENGTH": 12800, 
    "N_CLASS": 26, 
    "USE_NO_LOAD": False, 
    "USE_HAND_AUGMENTATION": False,
    "MARGIN_RATIO": 0.15, 
    "DATASET_PATH": "drive/MyDrive/Scattering_Novo/dataset_original/Synthetic_Full_iHall.hdf5",
    "TRAIN_SIZE": 0.9,
    "FOLDER_PATH": "drive/MyDrive/DeSpaWN-main/extracted_features/without_data_augmentation/Full_Dataset/", 
    "FOLDER_DATA_PATH": "drive/MyDrive/DeSpaWN-main/extracted_features/without_data_augmentation/Full_Dataset/", 
    "FEATURES_FILE_NAME": "features.mat",
    "N_EPOCHS_TRAINING": 500,
    "PERCENTUAL": [1],
    "INITIAL_EPOCH": 0,
    "TOTAL_MAX_EPOCHS": 5000,
    "SNRdb": None # Nível de ruído em db
}

def freeze(model, task_name='classification'):
    for layer in model.layers:
        if task_name in layer.name:
            layer.trainable = True
        else:
            layer.trainable = False

    for layer in model.layers:
        print(layer.name, layer.trainable)

    return model

def calculating_class_weights(y_true):
    '''
        Source: https://stackoverflow.com/questions/48485870/multi-label-classification-with-class-weights-in-keras
    '''
    from sklearn.utils.class_weight import compute_class_weight
    number_dim = np.shape(y_true)[1]
    weights = np.empty([number_dim, 2])
    for i in range(number_dim):
        weights[i] = compute_class_weight(class_weight='balanced', classes=[0.,1.], y=y_true[:, i])
    return weights


def reduce_dataset(X_all,ydet_all,ytype_all,yclass_all,percentual):
    import numpy as np
    max_index = int(percentual*X_all.shape[0])
    np.random.seed(100)
    index = np.random.randint(max_index,size=(max_index-1))
    X_all = X_all[index]
    ydet_all = ydet_all[index]
    ytype_all = ytype_all[index]
    yclass_all = yclass_all[index]

    return X_all,ydet_all,ytype_all,yclass_all

ngrids = configs["N_GRIDS"]
signalBaseLength = configs["SIGNAL_BASE_LENGTH"]
trainSize = configs["TRAIN_SIZE"]
folderDataPath = configs["FOLDER_DATA_PATH"]
 
dataHandler = DataHandler(configs)

if not os.path.isfile(folderDataPath + "data.p"):
    print("Sorted data not found, creating new file...")
    x, ydet, yclass, ytype, ygroup = dataHandler.loadData(hand_augmentation=configs["USE_HAND_AUGMENTATION"], SNR=configs["SNRdb"])
    print("Data loaded")

    data_mskf = MultilabelStratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    strat_classes = np.max(yclass, axis=1)
    train_index, test_index = next(data_mskf.split(x, strat_classes))

    y_train = {
        "detection": ydet[train_index], 
        "type": ytype[train_index], 
        "classification": yclass[train_index], 
        "group": ygroup[train_index]
    }
    
    y_test = {
        "detection": ydet[test_index], 
        "type": ytype[test_index], 
        "classification": yclass[test_index], 
        "group": ygroup[test_index]
    }
    
    dict_data = {
        "x_train": x[train_index], 
        "x_test": x[test_index], 
        "y_train": y_train, 
        "y_test": y_test
    }

    print("Data sorted")

    try:
        os.mkdir(folderDataPath)
    except:
        pass

    pickle.dump(dict_data, open(folderDataPath + "data.p", "wb"))
    print("Data stored")
else:
    dict_data = pickle.load(open(folderDataPath + "data.p", "rb"))



modelHandler = ModelHandler(configs)
 
X_all = dict_data["x_train"]
ydet_all = dict_data["y_train"]["detection"]
ytype_all = dict_data["y_train"]["type"]
yclass_all = dict_data["y_train"]["classification"]

if configs["PERCENTUAL"][0]!=1:
        X_all,ydet_all,ytype_all,yclass_all = reduce_dataset(X_all,ydet_all,ytype_all,yclass_all,configs["PERCENTUAL"][0])

print(X_all.shape)
print(dict_data["x_test"].shape)


Using TensorFlow backend.


(7575, 16640, 1)
(841, 16640, 1)


## Extraction Functions Definitions

## Without Grids

In [ ]:
def extract_DeSpaWN(input_signal, epochs, win_size=2000, mode = 'PerLayer'):
    """
    Title: Fully Learnable Deep Wavelet Transform for Unsupervised Monitoring of High-Frequency Time Series
    ------          (DeSpaWN)
    Description: 
    --------------
    Toy script to showcase the deep neural network DeSpaWN.
    Please cite the corresponding paper:
              Michau, G., Frusque, G., & Fink, O. (2022). 
              Fully learnable deep wavelet transform for unsupervised monitoring of high-frequency time series. 
              Proceedings of the National Academy of Sciences, 119(8).
              
    Version: 1.0
    --------
    
    @author:  Dr. Gabriel Michau,
    --------  Chair of Intelligent Maintenance Systems
              ETH Zürich
    
    Created on 15.01.2022
    
    Licence:
    ----------
    MIT License
    
    Copyright (c) 2022 Dr. Gabriel Michau
    
    Permission is hereby granted, free of charge, to any person obtaining a copy
    of this software and associated documentation files (the "Software"), to deal
    in the Software without restriction, including without limitation the rights
    to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
    copies of the Software, and to permit persons to whom the Software is
    furnished to do so, subject to the following conditions:
    
    The above copyright notice and this permission notice shall be included in all
    copies or substantial portions of the Software.
    
    THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
    IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
    FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
    AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
    LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
    OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
    SOFTWARE.
    
    """
    # Usual packages
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    
    import tensorflow as tf # designed with 2.1.0 /!\ models output changes with tf>2.2.0
    import tensorflow.keras as keras
    
    from lib import despawn
    
    
    signal = input_signal
    # Load a toy time series data to run DeSPAWN
    #signal = pd.read_csv("/Users/evertonluizdeaguiar/Documents/Doutorado_PC_Linux/Artigo_Sensors_2023/Metodos_Para_Testar/DeSpaWN-main/monthly-sunspots.csv")
    lTrain = win_size # length of the training section
    # Normalize and insert two additional axes
    #signalT = ((signal['Sunspots']-signal['Sunspots'].mean())/signal['Sunspots'].std()).values[np.newaxis,:,np.newaxis,np.newaxis]
    #signalT = ((signal-signal.mean())/signal.std()).values[np.newaxis,:,np.newaxis,np.newaxis]
    signalT = ((signal-signal.mean())/signal.std())[np.newaxis,:,np.newaxis,np.newaxis]

    signal = signalT[:,:lTrain,:,:]
    
    # Number of decomposition level is max log2 of input TS
    level = np.floor(np.log2(signal.shape[1])).astype(int)
    # Train hard thresholding (HT) coefficient?
    trainHT = True
    # Initialise HT value
    initHT = 0.3
    # Which loss to consider for wavelet coeffs ('l1' or None)
    lossCoeff='l1'
    # Weight for sparsity loss versus residual?
    lossFactor = 1.0
    # Train wavelets? (Trainable kernels)
    kernTrainable = True
    # Which training mode?
    # cf (https://arxiv.org/pdf/2105.00899.pdf -- https://doi.org/10.1073/pnas.2106598119) [Section 4.4 Ablation Study]
    #   CQF => learn wavelet 0 infer all other kernels from the network
    #   PerLayer => learn one wavelet per level, infer others
    #   PerFilter => learn wavelet + scaling function per level + infer other
    #   Free => learn everything
    #mode = 'PerLayer' # QMF PerLayer PerFilter Free
    
    # Initialise wavelet kernel (here db-4)
    kernelInit = np.array([-0.010597401785069032, 0.0328830116668852, 0.030841381835560764, -0.18703481171909309,
                               -0.027983769416859854, 0.6308807679298589, 0.7148465705529157, 0.2303778133088965])
    
    
    epochs = configs["N_EPOCHS_TRAINING"]
    verbose = 0
    
    # Set sparsity (dummy) loss:
    def coeffLoss(yTrue,yPred):
        return lossFactor*tf.reduce_mean(yPred,keepdims=True)
    # Set residual loss:
    def recLoss(yTrue,yPred):
        return tf.math.abs(yTrue-yPred)
    
    keras.backend.clear_session()
    # generates two models: 
    #      model1 outputs the reconstructed signals and the loss on the wavelet coefficients
    #      model2 outputs the reconstructed signals and wavelet coefficients
    model1,model2 = despawn.createDeSpaWN(inputSize=None, kernelInit=kernelInit, kernTrainable=kernTrainable, level=level, lossCoeff=lossCoeff, kernelsConstraint=mode, initHT=initHT, trainHT=trainHT)
    opt = keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Nadam")
    # For the training we only use model1
    model1.compile(optimizer=opt, loss=[recLoss, coeffLoss])
    # the sparsity term has no ground truth => just input an empty numpy array as ground truth (anything would do, in coeffLoss, yTrue is not called)
    H = model1.fit(signal,[signal,np.empty((signal.shape[0]))], epochs=epochs, verbose=verbose)
    
    # Returning 
    
    
    rec_signal, b, coefs = model2.predict(signal)
    
    #coefs = model2.get_weights();
    #coefs = [np.array(p) for p in coefs]
    coefs_out = np.zeros(len(coefs))
    
    for i in range(len(coefs)): # for every level
        coefs_temp = np.reshape(coefs[i], (coefs[i].shape[1],) )
        coefs_temp = np.mean(coefs_temp) # Take the average of this level
        coefs_out[i] = coefs_temp # Store the last mean
        
    coefs_out = coefs_out[::-1] # Reverse the order, because the original output is inverted distributed
    del model1, model2, H, rec_signal, b, coefs  
    return coefs_out


## With Grids

In [ ]:
def extract_DeSpaWN_grids4(input_signal, epochs, win_size=2000, mode = 'PerLayer'):

    # Usual packages
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    
    import tensorflow as tf # designed with 2.1.0 /!\ models output changes with tf>2.2.0
    import tensorflow.keras as keras
    
    from lib import despawn
    
    
    signal = input_signal
    # Load a toy time series data to run DeSPAWN
    #signal = pd.read_csv("/Users/evertonluizdeaguiar/Documents/Doutorado_PC_Linux/Artigo_Sensors_2023/Metodos_Para_Testar/DeSpaWN-main/monthly-sunspots.csv")
    lTrain = win_size # length of the training section
    # Normalize and insert two additional axes
    #signalT = ((signal['Sunspots']-signal['Sunspots'].mean())/signal['Sunspots'].std()).values[np.newaxis,:,np.newaxis,np.newaxis]
    #signalT = ((signal-signal.mean())/signal.std()).values[np.newaxis,:,np.newaxis,np.newaxis]
    signalT = ((signal-signal.mean())/signal.std())[np.newaxis,:,np.newaxis,np.newaxis]

    signal = signalT[:,:lTrain,:,:]
    
    # Number of decomposition level is max log2 of input TS
    level = np.floor(np.log2(signal.shape[1])).astype(int) - 4
    # Train hard thresholding (HT) coefficient?
    trainHT = True
    # Initialise HT value
    initHT = 0.3
    # Which loss to consider for wavelet coeffs ('l1' or None)
    lossCoeff='l1'
    # Weight for sparsity loss versus residual?
    lossFactor = 1.0
    # Train wavelets? (Trainable kernels)
    kernTrainable = True
    # Which training mode?
    # cf (https://arxiv.org/pdf/2105.00899.pdf -- https://doi.org/10.1073/pnas.2106598119) [Section 4.4 Ablation Study]
    #   CQF => learn wavelet 0 infer all other kernels from the network
    #   PerLayer => learn one wavelet per level, infer others
    #   PerFilter => learn wavelet + scaling function per level + infer other
    #   Free => learn everything
    #mode = 'PerLayer' # QMF PerLayer PerFilter Free
    
    # Initialise wavelet kernel (here db-4)
    kernelInit = np.array([-0.010597401785069032, 0.0328830116668852, 0.030841381835560764, -0.18703481171909309,
                               -0.027983769416859854, 0.6308807679298589, 0.7148465705529157, 0.2303778133088965])
    
    
    epochs = configs["N_EPOCHS_TRAINING"]
    verbose = 0
    
    # Set sparsity (dummy) loss:
    def coeffLoss(yTrue,yPred):
        return lossFactor*tf.reduce_mean(yPred,keepdims=True)
    # Set residual loss:
    def recLoss(yTrue,yPred):
        return tf.math.abs(yTrue-yPred)
    
    keras.backend.clear_session()
    # generates two models: 
    #      model1 outputs the reconstructed signals and the loss on the wavelet coefficients
    #      model2 outputs the reconstructed signals and wavelet coefficients
    model1,model2 = despawn.createDeSpaWN_v2(inputSize=None, kernelInit=kernelInit, kernTrainable=kernTrainable, level=level, lossCoeff=lossCoeff, kernelsConstraint=mode, initHT=initHT, trainHT=trainHT)
    opt = keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Nadam")
    # For the training we only use model1
    model1.compile(optimizer=opt, loss=[recLoss, coeffLoss])
    # the sparsity term has no ground truth => just input an empty numpy array as ground truth (anything would do, in coeffLoss, yTrue is not called)
    H = model1.fit(signal,[signal,np.empty((signal.shape[0]))], epochs=epochs, verbose=verbose)
    
    # Returning 
    
    
    gint, g, h = model2.predict(signal)
    
    #coefs = model2.get_weights();
    #coefs = [np.array(p) for p in coefs]
    #coefs_out = np.zeros(len(coefs))
    coefs_out = np.array([])

       
        #print(g_temp)
    gint = gint[0].reshape((gint[0].shape[0],)) # fitting the size

# calculate how big each part should be, rounding up
    part_size = int(np.ceil(len(gint) / 5))

# calculate how many extra elements need to be added to the array
    extra_elements = part_size * 5 - len(gint)

# add the extra elements distributed to both sides of the array
    gint = np.pad(gint, (int(np.ceil(extra_elements/2)), extra_elements-int(np.ceil(extra_elements/2))), mode='constant')

# divide the array into 5 equal parts
    gint = np.reshape(gint, (5, part_size))

# Calculate the average of each grid

    gint = np.mean(gint, axis=1) 
    gint = gint.reshape((5,-1)) # will generate a shape array (5,1), for this level





    for i in range(len(h)): # for every level
        
        
        # For detail coefs

        h_temp = h[i]
        h_temp = h[i].reshape((h[i].shape[1],)) # fitting the size

# calculate how big each part should be, rounding up
        part_size = int(np.ceil(len(h_temp) / 5))

# calculate how many extra elements need to be added to the array
        extra_elements = part_size * 5 - len(h_temp)

# add the extra elements distributed to both sides of the array
        h_temp = np.pad(h_temp, (int(np.ceil(extra_elements/2)), extra_elements-int(np.ceil(extra_elements/2))), mode='constant')

# divide the array into 5 equal parts
        h_temp = np.reshape(h_temp, (5, part_size))

# Calculate the average of each grid

        h_temp = np.mean(h_temp, axis=1) 
        h_temp = h_temp.reshape((5,-1)) # will generate a shape array (5,1), for this level


        
        ##coefs_temp = h_temp + g_temp # sum both high and low frequency average values per-grid
        coefs_temp = h_temp  # sum both high and low frequency average values per-grid
        coefs_temp2 = np.zeros_like(coefs_temp)

        coefs_temp2[0] = coefs_temp[1]-coefs_temp[0] # F_{1,level} = g_{2,level} - g_{1,level}
        coefs_temp2[1] = coefs_temp[2]-coefs_temp[1] # F_{2,level} = g_{3,level} - g_{2,level}
        coefs_temp2[2] = coefs_temp[3]-coefs_temp[2] # F_{3,level} = g_{4,level} - g_{3,level}
        coefs_temp2[3] = coefs_temp[4]-coefs_temp[3] # F_{4,level} = g_{5,level} - g_{4,level}
        coefs_temp2[4] = coefs_temp[4]               # F_{5,level} = g_{5,level} 

        coefs_temp = coefs_temp2

        #del coefs_temp2, g_temp, h_temp
        del coefs_temp2, h_temp

        if i==0:
          coefs_out = coefs_temp
        else:
          coefs_out = np.append(coefs_out, coefs_temp, axis=1) # apensamos no axis=1, que é a coluna. A ideia é resultar em uma estrutura de shape (5, level), em que level é número de níveis

        del coefs_temp
        #

        #coefs_temp = np.mean(coefs_temp, axis=1) # Take the average of this level
        #coefs_out[i] = coefs_temp # Store the last mean

    coefs_out = np.concatenate((gint,coefs_out), axis=1) # Concatenating the approximation with the details


    coefs_out = coefs_out.reshape((1,coefs_out.shape[0],coefs_out.shape[1])) # reshape to (1,n_grids,n_feats), where n_feats = levels+1

    #coefs_out = coefs_out[::-1] # Reverse the order, because the original output is inverted distributed
    del model1, model2, H, h
    return coefs_out


## Testing the new function

In [ ]:
X_test = dict_data["x_test"]
ytype_test = dict_data["y_test"]["type"]
yclass_test = dict_data["y_test"]["classification"]
ydet_test = dict_data["y_test"]["detection"]


In [ ]:
yclass_test.shape

(841, 5, 26)

## Training Features Extraction

In [ ]:
saving_size = 100 # Create a file every saving_size examples
cont_ini = 46 # Initial file index

# For every sample in the training dataset
features = np.array([])
cont = cont_ini
k = 0
for i in range(cont_ini*saving_size,X_all.shape[0]):

    if k==0:
      features = extract_DeSpaWN_grids4(input_signal=X_all[i,:,0], epochs=configs["N_EPOCHS_TRAINING"], win_size=2**14, mode = 'PerLayer')
      print("Extracting Training Features " + str(i) + ' / ' + str(X_all.shape[0]))
      print("Sample " + str(k) + " of file " + str(cont))
    else:
      features = np.append(features, extract_DeSpaWN_grids4(input_signal=X_all[i,:,0], epochs=configs["N_EPOCHS_TRAINING"], win_size=2**14, mode = 'PerLayer'), axis=0 )
      print("Extracting Training Features " + str(i) + ' / ' + str(X_all.shape[0]))
      print("Sample " + str(k) + " of file " + str(cont))
    if k==saving_size-1:
      k=0
      # Save partials
      print("Extraction Finished! Now Saving.")
      sio.savemat(configs["FOLDER_PATH"]+ "features_train_grids4" + str(cont) + ".mat", {'arr': features})
      cont = cont+1
      features = np.array([])
      # Saving features in .mat
    else:
          k=k+1
# for the last file (file 75)
print("Extraction Finished! Saving the last training file.")
sio.savemat(configs["FOLDER_PATH"]+ "features_train_grids4"+ str(cont) + ".mat", {'arr': features})


# Saving Entire training labels
#sio.savemat(configs["FOLDER_PATH"]+ "ytype_train_"+ ".mat", {'arr': ytype_all})
#sio.savemat(configs["FOLDER_PATH"]+ "yclass_train_" + ".mat", {'arr': yclass_all})
#sio.savemat(configs["FOLDER_PATH"]+ "ydet_train_" + ".mat", {'arr': ydet_all})



Extracting Training Features 4600 / 7575
Sample 0 of file 46
Extracting Training Features 4601 / 7575
Sample 1 of file 46
Extracting Training Features 4602 / 7575
Sample 2 of file 46
Extracting Training Features 4603 / 7575
Sample 3 of file 46


Extracting Training Features 4604 / 7575
Sample 4 of file 46


A saída de streaming foi truncada nas últimas 5000 linhas.
Extracting Training Features 5088 / 7575
Sample 88 of file 50
Extracting Training Features 5089 / 7575
Sample 89 of file 50
Extracting Training Features 5090 / 7575
Sample 90 of file 50
Extracting Training Features 5091 / 7575
Sample 91 of file 50
Extracting Training Features 5092 / 7575
Sample 92 of file 50
Extracting Training Features 5093 / 7575
Sample 93 of file 50
Extracting Training Features 5094 / 7575
Sample 94 of file 50
Extracting Training Features 5095 / 7575
Sample 95 of file 50
Extracting Training Features 5096 / 7575
Sample 96 of file 50
Extracting Training Features 5097 / 7575
Sample 97 of file 50
Extracting Training Features 5098 / 7575
Sample 98 of file 50
Extracting Training Features 5099 / 7575
Sample 99 of file 50
Extraction Finished! Now Saving.
Extracting Training Features 5100 / 7575
Sample 0 of file 51
Extracting Training Features 5101 / 7575
Sample 1 of file 51
Extracting Training Features 5102 / 7575
S

## Testing Features Extraction

In [ ]:
saving_size = 100 # Create a file every saving_size examples
cont_ini = 0 # Initial file index

# For every sample in the training dataset
features = np.array([])
cont = cont_ini
k = 0
for i in range(cont_ini*saving_size,X_test.shape[0]):
#for i in range(2,4):

    if k==0:
      features = extract_DeSpaWN_grids4(input_signal=X_test[i,:,0], epochs=configs["N_EPOCHS_TRAINING"], win_size=2**14, mode = 'PerLayer')
      print("Extracting Testing Features " + str(i) + ' / ' + str(X_test.shape[0]))
      print("Sample " + str(k) + " of file " + str(cont))

    else:
      feats = extract_DeSpaWN_grids4(input_signal=X_test[i,:,0], epochs=configs["N_EPOCHS_TRAINING"], win_size=2**14, mode = 'PerLayer')
      features = np.append(features, feats, axis=0 )
      print("Extracting Testing Features " + str(i) + ' / ' + str(X_test.shape[0]))
      print("Sample " + str(k) + " of file " + str(cont))

    if k==saving_size-1:
      k=0
      # Save partials
      print("Extraction Finished! Now Saving.")
      sio.savemat(configs["FOLDER_PATH"]+ "features_train_grids4" + str(cont) + ".mat", {'arr': features})
      cont = cont+1
      features = np.array([])
      # Saving features in .mat
    else:
          k=k+1
# for the last file (file 75)

print("Extraction Finished! Saving the last testing file.")
sio.savemat(configs["FOLDER_PATH"]+ "features_train_grids4"+ str(cont) + ".mat", {'arr': features})


# Saving Entire training labels
sio.savemat(configs["FOLDER_PATH"]+ "ytype_test_"+ ".mat", {'arr': ytype_test})
sio.savemat(configs["FOLDER_PATH"]+ "yclass_test_" + ".mat", {'arr': yclass_test})
sio.savemat(configs["FOLDER_PATH"]+ "ydet_test_" + ".mat", {'arr': ydet_test})



Extracting Testing Features 0 / 841
Sample 0 of file 0
Extracting Testing Features 1 / 841
Sample 1 of file 0
Extracting Testing Features 2 / 841
Sample 2 of file 0
Extracting Testing Features 3 / 841
Sample 3 of file 0
Extracting Testing Features 4 / 841
Sample 4 of file 0
Extracting Testing Features 5 / 841
Sample 5 of file 0
Extracting Testing Features 6 / 841
Sample 6 of file 0
Extracting Testing Features 7 / 841
Sample 7 of file 0
Extracting Testing Features 8 / 841
Sample 8 of file 0
Extracting Testing Features 9 / 841
Sample 9 of file 0
Extracting Testing Features 10 / 841
Sample 10 of file 0
Extracting Testing Features 11 / 841
Sample 11 of file 0
Extracting Testing Features 12 / 841
Sample 12 of file 0
Extracting Testing Features 13 / 841
Sample 13 of file 0
Extracting Testing Features 14 / 841
Sample 14 of file 0
Extracting Testing Features 15 / 841
Sample 15 of file 0
Extracting Testing Features 16 / 841
Sample 16 of file 0
Extracting Testing Features 17 / 841
Sample 17 of f

In [ ]:
print(features.shape)

(2, 5, 11)


In [ ]:
# Saving Entire training labels
sio.savemat(configs["FOLDER_PATH"]+ "ytype_test_"+ ".mat", {'arr': ytype_test})
sio.savemat(configs["FOLDER_PATH"]+ "yclass_test_" + ".mat", {'arr': yclass_test})
sio.savemat(configs["FOLDER_PATH"]+ "ydet_test_" + ".mat", {'arr': ydet_test})